In [1]:
from typing import List, Optional
import pandas as pd; pd.set_option('display.max_columns', 100)
from pandas.core.common import flatten
import numpy as np
import shap
from datetime import datetime

from tqdm.notebook import tqdm

import re
import os

from functools import partial
from scipy.stats import mode
from scipy.spatial.distance import cdist

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns
# import plotly.express as px

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler,PowerTransformer, FunctionTransformer, OneHotEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, cohen_kappa_score, log_loss, f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFE, RFECV
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import CalibrationDisplay
from sklearn.inspection import PartialDependenceDisplay, permutation_importance
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.manifold import TSNE
# import optuna
import catboost as cb
import xgboost as xgb

from View import MyClass
mc = MyClass()

import warnings
warnings.filterwarnings('ignore')

In [2]:
params_cb = {
        "n_estimators": 1993,
        "learning_rate": 0.009906548674016816,
        "subsample": 0.5000411100811685,
#         'colsample_bylevel': 0.5853177112754828,
        "loss_function": "Logloss",
        "eval_metric": "AUC",
        "task_type": "CPU",
        #"max_bin": 20,
        "verbose": False,
        "max_depth": 5,
        "l2_leaf_reg": 20,
        "early_stopping_rounds": 100,
#         "thread_count": 6,
        # "random_seed": 42,
        "class_weights" : (1, 2),
        "random_state": 42
        }
params_lgbm = {"objective":'binary',
          "n_estimators" : 3000,
                             "max_depth" : 6,
                             "learning_rate" : 0.01,
                             "num_leaves" : 20,
                             "reg_alpha" : 4,
                             "reg_lambda" : 4,
                             "subsample" : 0.7,
                             "colsample_bytree" : 0.7,
                             "verbose" : -100,
                              "min_data_in_bin": 1,
                              "min_data_in_leaf" : 1
}
params_xgb = {
            # "objective":'binary',
            'max_delta_step': 3,
            'min_child_weight': 2,
            "n_estimators" : 2466,
            'alpha': 2,
             'colsample_bytree': 0.7,
             'eta': 0.003419782719339618,
             'gamma': 1,
             'lambda': 1,
             'max_depth': 8,
             'subsample': 0.31098721316506717
}

In [3]:
print(mc.__doc__)

Визуализация и предобработка данных
_________________________________________________
* Метод "view_csv" принимает DataFrame и выводит данные о нем
возвращает categorical, numerical

* Метод "stat_frame" принимает DataFrame и возвращает DataFrame со статистиками

* Метод corr(data, target):
data: DataFrame
target : str

* Графики
* Метод connect_sign(train, test, columns) Определяет похожи ли признаки в train и test
train, test: DataFrame
columns: list( список колонок без целевой переменной)

* Метод distribution_sign(train, numerical, target_name='target', log=False): распределение признаков
train: DataFrame
numerical: признаки числовые
target_name: str имя целевой переменной
log: bool если нужно логарифмировать признаки

* Метод similarity_sign(data, data_test, numericaal): похожесть признаков и вычисление p-value
data, data_test: DataFrame
numericaal: list числовые признаки

Работа с данными
__________________________________________________

* Метод transformer(train, test, cont_co

In [4]:
os.listdir('data')

['sample_submission.csv', 'test.csv', 'train.csv']

In [5]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sub = pd.read_csv('data/sample_submission.csv')

In [6]:
cat, num = mc.view_csv(train)
cat_t, num_t = mc.view_csv(test)

rows: 165034, cols: 14

Категориальные признаки: 
Всего категориальных признаков: 3
surname уникальных значений: 2797
geography уникальных значений: 3, ['France' 'Spain' 'Germany']
gender уникальных значений: 2, ['Male' 'Female']

Числовые признаки: 

Всего числовых признаков: 11
['hascrcard', 'estimatedsalary', 'numofproducts', 'exited', 'id', 'customerid', 'tenure', 'isactivemember', 'creditscore', 'balance', 'age']
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
В данных пропусков нету
rows: 110023, cols: 13

Категориальные признаки: 
Всего категориальных признаков: 3
surname уникальных значений: 2708
geography уникальных значений: 3, ['France' 'Germany' 'Spain']
gender уникальных значений: 2, ['Female' 'Male']

Числовые призна

In [7]:
train = train.drop("surname", axis=1)
test = test.drop("surname", axis=1)

In [8]:
aggs = ['geography', 'gender']

In [11]:
train, test = mc.encoder(train, test, aggs)

тип кодирования 1: OneHot, 2: LabelEncoder, 3: frequency 3


Готово, размер:  (165034, 20) (110023, 19)


In [12]:
train = train.drop(aggs, axis=1)
test = test.drop(aggs, axis=1)

In [13]:
train.to_csv("data/train_n.csv")
test.to_csv("data/test_n.csv")

In [13]:
X = train.drop(columns = ['exited'], axis = 1)
Y = train['exited']

test_cv = test

In [14]:
train.columns = train.columns.astype(str)
test.columns = test.columns.astype(str)

In [17]:
ens_cv_scores, ens_preds = list(), list()
hill_ens_cv_scores, hill_ens_preds =  list(), list()

sk = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 1, random_state = 42)
for i, (train_idx, test_idx) in enumerate(sk.split(X, Y)):

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]
    
    print('----------------------------------------------------------')
    
    ########
    ## RF ##
    ########

    RF_md = RandomForestClassifier(n_estimators = 2000, 
                                   max_depth = 6,
                                   min_samples_split = 15,
                                   min_samples_leaf = 10).fit(X_train, Y_train)
    
    RF_pred = RF_md.predict_proba(X_test)[:, 1]
    RF_score = roc_auc_score(Y_test, RF_pred)

    print('Fold', i, '==> RF oof ROC-AUC score is ==>', RF_score)

    RF_pred_test = RF_md.predict_proba(test_cv)[:, 1]
    
    #################
    ## Extra Trees ##
    #################

    ET_md = ExtraTreesClassifier(n_estimators = 2000, 
                                 max_depth = 6,
                                 min_samples_split = 15,
                                 min_samples_leaf = 10).fit(X_train, Y_train)

    ET_pred = ET_md.predict_proba(X_test)[:, 1]
    ET_score = roc_auc_score(Y_test, ET_pred)

    print('Fold', i, '==> ET oof ROC-AUC score is ==>', ET_score)

    ET_pred_test = ET_md.predict_proba(test_cv)[:, 1]

    ##########################
    ## HistGradientBoosting ##
    ##########################

    # hist_md = make_pipeline(StandardScaler(),
    #                        HistGradientBoostingClassifier(l2_regularization = 0.01,
    #                                          early_stopping = False,
    #                                          learning_rate = 0.01,
    #                                          max_iter = 8000,
    #                                          max_depth = 6,
    #                                          max_bins = 20,
    #                                          min_samples_leaf = 15,
    #                                          max_leaf_nodes = 10)).fit(X_train, Y_train)
    
    hist_md = HistGradientBoostingClassifier(l2_regularization = 0.01,
                                             early_stopping = False,
                                             learning_rate = 0.01,
                                             max_iter = 500,
                                             max_depth = 5,
                                             max_bins = 255,
                                             min_samples_leaf = 15,
                                             max_leaf_nodes = 10).fit(X_train, Y_train)
    
    hist_pred = hist_md.predict_proba(X_test)[:, 1]
    hist_score = roc_auc_score(Y_test, hist_pred)

    print('Fold', i, '==> Hist oof ROC-AUC score is ==>', hist_score)  

    hist_pred_test = hist_md.predict_proba(test_cv)[:, 1]

    ##########
    ## LGBM ##
    ##########

    LGBM_md = LGBMClassifier(objective = 'binary',
                             n_estimators = 2000,
                             max_depth = 6,
                             learning_rate = 0.01,
                             num_leaves = 20,
                             reg_alpha = 3,
                             reg_lambda = 3,
                             subsample = 0.7,
                             force_row_wise=True,
                             colsample_bytree = 0.7).fit(X_train, Y_train)

    lgb_pred = LGBM_md.predict_proba(X_test)[:, 1]
    lgb_score = roc_auc_score(Y_test, lgb_pred)

    print('Fold', i, '==> LGBM oof ROC-AUC score is ==>', lgb_score) 

    lgb_pred_test = LGBM_md.predict_proba(test_cv)[:, 1]

    #########
    ## XGB ##
    #########

    XGB_md = XGBClassifier(objective = 'binary:logistic',
                           tree_method = 'hist',
                           colsample_bytree = 0.7, 
                           gamma = 2, 
                           learning_rate = 0.01, 
                           max_depth = 6, 
                           min_child_weight = 10, 
                           n_estimators = 2000, 
                           subsample = 0.7).fit(X_train, Y_train)

    xgb_pred = XGB_md.predict_proba(X_test)[:, 1]
    xgb_score = roc_auc_score(Y_test, xgb_pred)

    print('Fold', i, '==> XGB oof ROC-AUC score is ==>', xgb_score)

    xgb_pred_test = XGB_md.predict_proba(test_cv)[:, 1]

    ##############
    ## CatBoost ##
    ##############

    Cat_md = CatBoostClassifier(loss_function = 'Logloss',
                                iterations = 2000,
#                                 max_bin = 20,
                                learning_rate = 0.01,
                                depth = 6,
                                random_strength = 0.5,
                                bagging_temperature = 0.7,
                                border_count = 30,
                                l2_leaf_reg = 10,
                                verbose = False, 
                                task_type = 'CPU').fit(X_train, Y_train)

    cat_pred = Cat_md.predict_proba(X_test)[:, 1]
    cat_score = roc_auc_score(Y_test, cat_pred)

    print('Fold', i, '==> CatBoost oof ROC-AUC score is ==>', cat_score)

    cat_pred_test = Cat_md.predict_proba(test_cv)[:, 1]    
    
    ##############
    ## Ensemble ##
    ##############
    
    ens_pred_1 = (RF_pred + ET_pred + hist_pred + lgb_pred + xgb_pred + cat_pred) / 6
    ens_pred_2 = (RF_pred_test + ET_pred_test + hist_pred_test + lgb_pred_test + xgb_pred_test + cat_pred_test) / 6
    
    ens_score_fold = roc_auc_score(Y_test, ens_pred_1)
    ens_cv_scores.append(ens_score_fold)
    ens_preds.append(ens_pred_2)
    
    print('Fold', i, '==> Average Ensemble oof ROC-AUC score is ==>', ens_score_fold)
    
    ############################
    ## Hill Climbing Ensemble ##
    ############################
    
    x = pd.DataFrame({'RF': RF_pred,
                      'ET': ET_pred, 
                      'Hist': hist_pred, 
                      'LGBM': lgb_pred,
                      'XGB': xgb_pred,
                      'Cat': cat_pred})
    y = Y_test
        
    x_test = pd.DataFrame({'RF': RF_pred_test,
                           'ET': ET_pred_test, 
                           'Hist': hist_pred_test, 
                           'LGBM': lgb_pred_test,
                           'XGB': xgb_pred_test,
                           'Cat': cat_pred_test})
    
    hill_results = mc.hill_climbing(x, y, x_test)
    
    hill_ens_score_fold = roc_auc_score(y, hill_results[0])
    hill_ens_cv_scores.append(hill_ens_score_fold)
    hill_ens_preds.append(hill_results[1])

    print('Fold', i, '==> Hill Climbing Ensemble oof ROC-AUC score is ==>', hill_ens_score_fold)

----------------------------------------------------------
Fold 0 ==> RF oof ROC-AUC score is ==> 0.8792879725878232
Fold 0 ==> ET oof ROC-AUC score is ==> 0.8552482419398224
Fold 0 ==> Hist oof ROC-AUC score is ==> 0.8874158137446392
[LightGBM] [Info] Number of positive: 27937, number of negative: 104090
[LightGBM] [Info] Total Bins 1378
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211601 -> initscore=-1.315304
[LightGBM] [Info] Start training from score -1.315304
Fold 0 ==> LGBM oof ROC-AUC score is ==> 0.890118900458089
Fold 0 ==> XGB oof ROC-AUC score is ==> 0.8900231122094434
Fold 0 ==> CatBoost oof ROC-AUC score is ==> 0.8894864471155329
Fold 0 ==> Average Ensemble oof ROC-AUC score is ==> 0.8885076506688792
Fold 0 ==> Hill Climbing Ensemble oof ROC-AUC score is ==> 0.8903386758625146
----------------------------------------------------------
Fold 1 ==> RF oof ROC-AUC score is ==> 0

In [18]:
# ens_preds_test = pd.DataFrame(ens_preds).apply(np.mean, axis = 0)

# sub['smoking'] = ens_preds_test
# sub.to_csv('ens_sub.csv', index = False)

ens_preds_test = pd.DataFrame(hill_ens_preds).apply(np.mean, axis = 0)

# sub['smoking'] = ens_preds_test
# sub.to_csv('hill_sub.csv', index = False)

sub["Exited"] = ens_preds_test
if os.path.exists("submission") == False:
    os.mkdir("submission")
sub.to_csv("submission/hill_sub.csv", index=False)